# Travelling Salesman Problem with Time Windows (TSPTW) 
> Please note, this problem builds off of the Travelling Salesman Problem Example and adds the constraint of requiring the locations to be visited within specified time windows.

## Introduction
The goal of the Travelling Salesperson Problem (TSP) is to find the optimal route through a network which visits all stops.  The TSPTW adds the constraint that certain nodes stops can only be serviced at specified times.

> The Icepack TSPTW model defines the optimal route as the one completed in the shortest period of time (consistent with the academic definition).  Distance is not taken directly into account, although the quickest route is often closely related to the shortest. For more control over the details over the objective function feel free to have a look at the IVR7/8 examples.
that will result in a different solution.

## Requirements
The only difference in requirements between the TSP example and the TSPTW example is that a different `.proto` file is used (`tsptw-kcxbievqo879.proto`) to defined the model (`tsptw_kcxbievqo879_pb2.py`).


## Protocol Buffers (Protobuf)
If you're using a version of `protoc` < 3.6.0 feel free to regenerate the `tsptw_kcxbievqo879_pb2.py` file using
following commands:

`protoc --python_out=. problem.proto` 

`protoc --python_out=. tsptw-kcxbievqo879`

`problem_pb2` and `tsptw_kcxbievqo879_pb2` can then be imported and used to serialise data.

## Icepack Client Portal
The Icepack Client Portal can be found at https://portal.test.icepack.ai.

### API Key
Ensure that you have access to a key that has the **Travelling Salesman Problem Solver** enabled.  A personal token will need to generated if it has not already been done.

## Time Windows
Time windows are periods of time during which the visit must occur.  For the TSPTW model, each point is permitted only one window and is defined using the _windowStart_ and the _windowEnd_ parameters in the Geocode message.  

**Either both the windowStart and windowEnd should be set - or neither.** If you're looking to have an unbounded time on either the start or end please specify a zero or large value (like 10000), respectively. 

When using Road Network distance, the unit of measurment for time windows is **decimal minutes**, with 0 being defined as the time of leaving the first point to start the route.

## TSPTW Set Up
The Icepack TSP service only requires the locations to be visited and no tasks to be completed while at this location.  Each location requires an ID, and a pair of coordinates.  For earth-routing, x and y (in caretesian space) map to the location's longitude and latitude.

> Please check the regions currently covered [here](https://docs.scrudu.io/about/regions/#readout). If your geocodes fall outside this region we will assume a zero distance between points (and an unreasonable result will follow).

In addition, optional time windows can be added to each point.

This problem is set up in the exact same manner as the previous TSP Example apart from using `tsptw_kcxbievqo879_pb2` instead of `tsp_mcvfz472gty6_pb2` and adding time windows to the points.

We're going to:
1. Load the api-helper (see `apiHelper.py` for details)
2. Load the sample data. (`publist.csv`)
3. Build a model (`tsptw_kcxbievqo879_pb2`)
4. Submit it to the api. (`api.icepack.ai/vehicle-router/solve`)
5. Get the response. (using the response from 4.)
6. Plot the result (using `ipyleaflet`)


### Load the data
We're only going to load the first 10 points to illustrate this example (mostly because making up random data which is feasible and reasonable in a real-world sense is hard!)

In [ ]:
import pandas
df = pandas.read_csv('../sample_data/publist.csv').head(10) 
print(df.head())


In [ ]:
import numpy
rupper = 2500
df['WindowStart'] = numpy.random.uniform(low=0.0, high=rupper, size=df.shape[0])
df['WindowEnd'] = rupper + df['WindowStart'] #so that our time windows are ordered correctly in time i.e. start < end
df.head()


### Build the model
Now that we have some data to work with, we can go ahead and start building the model.

In [ ]:
exec(open('apiHelper.py').read()) # import some helper classes which we've written for you.

api = apiHelper(modelType="tsptw-kcxbievqo879")

sr = tsptw_kcxbievqo879_pb2.SolveRequest()
sr.solveType = 0

m = sr.model

# or add them individually 
for index, row in df.iterrows():
  l = tsptw_kcxbievqo879_pb2.Geocode()
  l.id = row['id']
  l.x = row['X']
  l.y = row['Y']
  l.windowStart = row['WindowStart']
  l.windowEnd = row['WindowEnd']
  m.points.append(l)  
m.distancetype = 1 # set the distance to road network (it is the default if we forget)

print(m) # that's what the model looks like, not too shabby.

### Submit the problem to the api
We can use the wrapper class to push the model to the api.


In [ ]:
reqId = api.Post(sr)
print(reqId) # if you want to see what the guid looks like.

### Get the response using the guid provided
Again, using the wrapper class to retrieve the response payload is pretty straight forward. The logs here illustate the work the api did for us in the background.

In [ ]:
sol = api.Get(reqId)

print(sol.tour) # which is the sequence of geocode-id's we should follow to do perform an optimal tour!
print(sol.arrivalTimes)

### TSP Solution
The solution contains the optimal tour, arrival times at each location and the edges between locations.

We can plot the tour as we did with the TSP response.

> If the _distancetype_ was set to use Euclidean or Haversine distance, then the _distance_ and _time_ output parameters will be identical. There will also be empty edge collections as there are no routes through the road network to generate :-) 

Tabulating the response from the api, we can see that each arrival is within the required windowStart and windowEnd.

In [ ]:
tour= pandas.DataFrame(list(sol.tour), columns=['tour'])
tour['ArrivalTime'] = list(sol.arrivalTimes)
tourpts = pandas.merge(tour,df,left_on='tour',right_on='id', how = 'left').drop(['id'], axis=1)
tourpts.head()


#### Visualising the response
If the road network has been requested, the _geometry_ outputs will be populated in the api response.  The _geometry_ of an edge provides the list of coordinates to plot an accurate path that is taken between two points.

If using Euclidean or Haversine distance, this field is not populated (since there is no path, just a segment between the start and end point).

In [ ]:
# so we want a list of list of pairs (y,x => lat, lon).
from ipyleaflet import Map, Polyline, Circle, LayerGroup

#stops
locs = list()
for index, p in tourpts.iterrows():
    circle = Circle()
    circle.location = (p['Y'], p['X'])
    circle.radius = 10
    circle.color = "green"
    locs.append(circle)

#routes
routes = list()
for i, e in enumerate(sol.edges):
    route = list()
    for j, p in enumerate(e.geometry):
        route.append([p.y, p.x])  #ipyleaflet neets this in lat-lon format - cartographically correct. Cartesianially incorrect :-) 
    routes.append(route)
    
center = [df['Y'][1],df['X'][1]] # just use the first point as the cetner.
m = Map(scroll_wheel_zoom=True, center=center, zoom=12)
m.add_layer(Polyline(locations = routes, 
                     color="blue", 
                     fill = False))
m.add_layer(LayerGroup(layers=(locs)))
m


### Arrival Times
The response provides the arrival time for each point on the tour. The arrival time will be set to the value of the _windowStart_ if arriving at a point before the window has started. In other words, early arrival is permitted in this model. To see the slack/tardy definitions in the documentation for the IVR7/8 models if you're looking for fine-grained control over these fields.



## Request Statistics
The statistics for all recent requests, including the solve duration, can be viewed in the dashboard of the Icepack Client Portal (https://portal.test.icepack.ai/dashboard).